In [28]:
import speech_recognition as sr
import pyttsx3

In [27]:
def transcribe_audio(wav_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(wav_file) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            return text
        except sr.UnknownValueError:
            return "Google Speech Recognition could not understand audio"
        except sr.RequestError as e:
            return f"Could not request results from Google Speech Recognition service; {e}"

def wav_to_text():
    wav_file = "./data/test.wav"  # Replace with your wav file path
    text = transcribe_audio(wav_file)
    print(text)

wav_to_text()

result2:
{   'alternative': [   {   'confidence': 0.92534178,
                           'transcript': 'the stale smell of old beer lingers '
                                         'it takes heat to bring out the odor '
                                         'a cold dip restores health and zest '
                                         'a salt pickle taste fine with ham '
                                         'tacos al pastor are my favorite a '
                                         'zestful food is the hot cross bun'},
                       {   'confidence': 0.92534178,
                           'transcript': 'the stale smell of old beer lingers '
                                         'it takes heat to bring out the odor '
                                         'a cold dip restores health and zest '
                                         'a salt pickle tastes fine with ham '
                                         'tacos al pastor are my favorite a '
             

In [29]:
import os

def text_to_speech_mac(text):
    os.system(f'say "{text}"')

text_to_speech_mac("Hello, this is a test of the macOS text-to-speech utility.")


/opt/anaconda3/lib/python3.7/site-packages/pyttsx3/drivers/nsss.py:12: ObjCSuperWarning: Objective-C subclass uses super(), but super is not objc.super
  class NSSpeechDriver(NSObject):


AttributeError: 'super' object has no attribute 'init'